In [1]:
import pandas as pd
import json

# Cargar el archivo CSV
file_path = r'C:/Users/Mateo Grisales/Downloads/Caudal_Medio_Dia_Historico_Nechi.csv'
data = pd.read_csv(file_path, sep=';', encoding='ISO-8859-1')

In [2]:
data.columns

Index(['year', 'month', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10',
       '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22',
       '23', '24', '25', '26', '27', '28', '29', '30', '31'],
      dtype='object')

In [3]:
melted_data = data.melt(id_vars=["year", "month"], var_name="day", value_name="Caudal")
melted_data.dropna(subset=["Caudal"], inplace=True)
melted_data['day'] = melted_data['day'].astype(int)
melted_data['Fecha'] = pd.to_datetime(melted_data[['year', 'month', 'day']])
melted_data.sort_values('Fecha', inplace=True)
melted_data['Fecha'] = melted_data['Fecha'].astype(str)
melted_data['Latitud'] = 8.030111111
melted_data['Longitud'] = -74.78555556

In [4]:
melted_data['day'] = melted_data['day'].astype(int)
melted_data['Fecha'] = pd.to_datetime(melted_data[['year', 'month', 'day']]) + pd.to_timedelta('00:00:00')

In [5]:
def df_to_geojson(df, properties, lat='Latitud', lon='Longitud'):
    geojson = {'type': 'FeatureCollection', 'features': []}
    for _, row in df.iterrows():
        feature = {
            'type': 'Feature',
            'properties': {},
            'geometry': {
                'type': 'Point',
                'coordinates': [row[lon], row[lat]]
            }
        }
        for prop in properties:
            if isinstance(row[prop], pd.Timestamp):
                # Convertir datetime a string en formato ISO para GeoJSON
                feature['properties'][prop] = row[prop].isoformat()
            else:
                feature['properties'][prop] = row[prop]
        geojson['features'].append(feature)
    return geojson


In [6]:
properties = ['Fecha', 'Caudal']
geojson_data = df_to_geojson(melted_data, properties)

# Guardar el GeoJSON
geojson_path = 'C:/Users/Mateo Grisales/Downloads/Caudal_Nechi.geojson'
with open(geojson_path, 'w') as f:
    json.dump(geojson_data, f)

print(f"GeoJSON file created at {geojson_path}")

GeoJSON file created at C:/Users/Mateo Grisales/Downloads/Caudal_Nechi.geojson
